###### Задание №4 состоит из двух пунктов (вместе засчитываются за одну задачу)

А. Практика. Написать программу для поиска ближайшей пары точек на плоскости методом сканирующей строки

Дано: набор точек, заданных координатами x и y. Для тестирования создайте набор из 1000 точек путем генерации псевдослучайных чисел.

Найти и вывести:

1.       Евклидово расстояние между ближайшими точками

2.       Номер первой точки пары и ее координаты

3.       Номер второй точки пары и ее координаты

4.       Число сделанных расчётов расстояний евклидова расстояния между парами точек в течение всего алгоритма (это число должно быть существенно меньше, чем количество всех пар точек)

 Б. Теория. Тест пересечения отрезка и треугольника в 3D.

На лекции рассказывался тест пересечения двух отрезков в 2D на основе функции orient2D или CCW. Надо построить аналогичный тест для отрезка прямой и треугольника в 3D на основе функции orient3D: https://darrensweeney.net/2016/08/31/determinant-predicates/

Тест должен определять сам факт пересечения.

In [18]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [19]:
import numpy as np
from random import randint
import copy
import time

In [20]:
# дано 500 пар точек (всего 1000) 
points = np.random.randint(-1000, 1000, (1000, 2))
print(np.shape(points), points[0], points[1])

(1000, 2) [ -41 -548] [ 879 -634]


###### Plane Sweep Algorithm

1. Сортируем список точек слева направо по оси x
2. Для каждой точки:
    1. Мы удаляем из кандидатов все точки, которые расположены дальше по оси х, чем текущие точки с минимальным расстоянием
    2. Мы добавляем всех кандидатов, которые расположены на минимальном расстоянии по оси y от текущей точки
    3. Мы проверяем минимальное расстояние с текущей точкой у всех кандидатов, прошедших предварительную проверку
    4. И, наконец, мы добавляем текущую точку в список кандидатов

In [21]:
# https://baptiste-wicht.com/posts/2010/04/closest-pair-of-point-plane-sweep-algorithm.html

In [22]:
def find_number_of_points(points, closestPair):
    
    index_0 = -1 
    index_1 = -1
    
    for i in range(np.shape(points)[0]):
            if points[i][0] == int(closestPair[0][0]):
                if points[i][1] == int(closestPair[0][1]):
                    index_0 = i
                    
            if points[i][0] == int(closestPair[1][0]):
                if points[i][1] == int(closestPair[1][1]):
                    index_1 = i
                    
    return index_0, index_1 

In [24]:
def naive(points):
    min_dist = 100000
    dist = -1
    closest_naive = np.zeros((2, 2))
    
    for point in points:
        for point_2 in points:
            if not np.array_equal(point, point_2):
                dist = np.linalg.norm(point - point_2) # посчитали расстояние
                if dist < min_dist:
                    min_dist = dist
                    closest_naive[0] = point
                    closest_naive[1] = point_2
                    
    return np.linalg.norm(closest_naive[0] - closest_naive[1]), closest_naive

In [23]:
def delete_candidates_from_array(sort_points, candidates, index):
    
    value = sort_points[index]
    find_index = -1
    
    for i in range(np.shape(candidates)[0] - 1):
        if candidates[i][0] == value[0]:
            if candidates[i][1] == value[1]:
                find_index = i
                break;
                
    candidates.pop(find_index)
    return candidates

def closest_pair(points):
    total = 0
    closest_points = np.zeros((2, 2))
    
    cols = [0]
    sort_points = points[np.lexsort(points.T[cols])] # отсортировали по x
    candidates = [] # задали пустой массив для кандидатов
    
    left_index = 0
    crt_min_dist = np.inf # текущее самое маленькое расстояние -- бесконечность
    
    for current_point in sort_points:
        while current_point[0] - sort_points[left_index][0] > crt_min_dist:
            if len(candidates) != 0: # если кандидаты не пустые
                for current_cand in candidates:
                    if int(current_cand[0]) == int(sort_points[left_index][0]): # по X-координате смотрим
                        if int(current_cand[1]) == int(sort_points[left_index][1]):
                            # удаляем кандидата из списка 
                            candidates = delete_candidates_from_array(sort_points, candidates, left_index) 
                            left_index += 1
                            break;
            else:
                break
        
        if len(candidates) != 0:
            for current_cand in candidates:
                if current_cand[1] > current_point[1] - crt_min_dist: # по Y-координате смотрим
                    if current_cand[1] < current_point[1] + crt_min_dist:
                        distance = np.linalg.norm(current_cand - current_point) # посчитали расстояние
                        total += 1 # посчитали расстояние -- прибавили
                if distance < crt_min_dist:
                    if distance != 0.0: # близкие точки, а не одна и та же точка
                        crt_min_dist = distance
                        closest_points[0] = current_point
                        closest_points[1] = current_cand
                    
        candidates.append(list(current_point))
        candidates = sorted(candidates, key=lambda y: y[1])
        
    return closest_points, total

In [28]:
start_time = time.time()
pair, total = closest_pair(points)
print('Plane Sweep алгоритм занимает секунд: ', (time.time() - start_time))
print('1. Евклидово расстояние между ближайшими точками, Plane Sweep алгоритм: ', distance)
print('2. Номер первой точки пары и её координаты: ', cols_0, pair[0])
print('3. Номер второй точки пары и её координаты: ', cols_1, pair[1])
print('4. Сколько раз посчитали евклидово расстояние: ', total)

distance = np.linalg.norm(pair[0] - pair[1])

print('\n')

start_time = time.time()
distance_naive, closest_naive = naive(points)
print('Наивный алгоритм занимает секунд: ', (time.time() - start_time))

cols_0, cols_1 = find_number_of_points(points, pair)

print('Евклидово расстояние между ближайшими точками, наивный алгоритм: ', distance_naive)
print('Координаты точек, найденные наивным алгоритмом: ', closest_naive[0], closest_naive[1])

Plane Sweep алгоритм занимает секунд:  0.04727911949157715
1. Евклидово расстояние между ближайшими точками, Plane Sweep алгоритм:  1.0
2. Номер первой точки пары и её координаты:  367 [-890.  -79.]
3. Номер второй точки пары и её координаты:  36 [-891.  -79.]
4. Сколько раз посчитали евклидово расстояние:  9


Наивный алгоритм занимает секунд:  13.876905918121338
Евклидово расстояние между ближайшими точками, наивный алгоритм:  1.0
Координаты точек, найденные наивным алгоритмом:  [-891.  -79.] [-890.  -79.]


In [29]:
# https://docs.google.com/presentation/d/1nuIlnRX39omk5khA_HPGUba4HKIDNr94-1UTQ5npN-A/edit#slide=id.p3

In [30]:
# http://algolist.manual.ru/maths/geom/intersect/linefacet3d.php

In [32]:
# https://stackoverflow.com/questions/44513525/testing-whether-a-3d-point-is-inside-a-3d-polyhedron

In [33]:
# https://arxiv.org/pdf/1810.01175.pdf стр 125